Read JSON files from Naval's Volume and load to Bronze schema

In [0]:
from pyspark.sql.functions import *

In [0]:
bronze_catalog='cgi_dev'
bronze_schema='swetha_bronze'

file_path_circuit="/Volumes/cgi_dev/naval/formula1/circuits.csv"
file_path_constructors="/Volumes/cgi_dev/naval/formula1/constructors.json"
file_path_drivers="/Volumes/cgi_dev/naval/formula1/drivers.json"
file_path_pitstop="/Volumes/cgi_dev/naval/formula1/pit_stops.json"

In [0]:
df_circuit=spark.read.csv(file_path_circuit,header=True,inferSchema=True).withColumn("ingest_date",current_timestamp()).withColumn("metadata",col("_metadata"))
df_constructors=spark.read.json(file_path_constructors).withColumn("ingest_date",current_timestamp()).withColumn("metadata",col("_metadata"))
df_drivers=spark.read.json(file_path_drivers).withColumn("ingest_date",current_timestamp()).withColumn("metadata",col("_metadata"))
df_pitstop=spark.read.option("multiline",True).json(file_path_pitstop).withColumn("ingest_date",current_timestamp()).withColumn("metadata",col("_metadata"))

#df_circuit.display()

In [0]:
#df_circuit.display()
#df_constructors.display()
#df_drivers.display()
#df_pitstop.display()

In [0]:
df_circuit.write.mode("overwrite").saveAsTable(f"{bronze_catalog}.{bronze_schema}.circuits")
df_constructors.write.mode("overwrite").saveAsTable(f"{bronze_catalog}.{bronze_schema}.constructors")
df_drivers.write.mode("overwrite").saveAsTable(f"{bronze_catalog}.{bronze_schema}.drivers")
df_pitstop.write.mode("overwrite").saveAsTable(f"{bronze_catalog}.{bronze_schema}.pitstops")

In [0]:
%sql
select "circuits",count(*) from cgi_dev.swetha_bronze.circuits
union
select "constructors",count(*) from cgi_dev.swetha_bronze.constructors
union
select "drivers",count(*) from cgi_dev.swetha_bronze.drivers
union
select "pitstop",count(*) from cgi_dev.swetha_bronze.pitstops;


In [0]:
df=spark.table("cgi_dev.swetha_bronze.drivers")

In [0]:
df_final=df\
.withColumn("forename",col("name.forename"))\
.withColumn("surname",col("name.surname"))\
.drop("name","metadata","ingestion_date")

In [0]:
df_final.write.saveAsTable("cgi_dev.swetha_silver.drivers")

In [0]:
%sql
select * from cgi_dev.swetha_silver.drivers limit 10;